In [50]:
# Установка необходимых библиотек
!pip install  faiss-cpu

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
import numpy as np

# 1. Загрузка датасета из файла cat-facts.txt
def load_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        facts = [line.strip() for line in f if line.strip()]
    return facts

dataset = load_dataset("/content/кошки.txt")
print(f"Загружено {len(dataset)} фактов о кошках.")

# 2. Создание поискового механизма (TF-IDF + FAISS)
class Retriever:
    def __init__(self, documents):
        self.documents = documents
        self.vectorizer = TfidfVectorizer()
        self.tfidf_matrix = self.vectorizer.fit_transform(documents)

        # Преобразование TF-IDF матрицы в формат, подходящий для FAISS
        tfidf_array = self.tfidf_matrix.toarray().astype('float32')
        self.index = faiss.IndexFlatL2(tfidf_array.shape[1])  # L2 расстояние
        self.index.add(tfidf_array)

    def retrieve(self, query, top_k=5):
        query_vector = self.vectorizer.transform([query]).toarray().astype('float32')
        distances, indices = self.index.search(query_vector, top_k)
        return [self.documents[i] for i in indices[0]]

# Инициализация поискового механизма
retriever = Retriever(dataset)

# 3. Настройка генеративной модели
class Generator:
    def __init__(self, model_name="t5-small"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def generate(self, context, question):
        input_text = f"question: {question} context: {context}"
        inputs = self.tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

# Инициализация генеративной модели
generator =  T5ForConditionalGeneration.from_pretrained("t5-small")

# 4. Объединение поиска и генерации
def rag_pipeline(question, retriever, generator, top_k=5):
    # Поиск релевантных документов
    retrieved_docs = retriever.retrieve(question, top_k=top_k)
    context = " ".join(retrieved_docs)

    # Генерация ответа
    response = generator.generate(context, question)
    return response, retrieved_docs

# 5. Тестирование системы
question = "сколько кошки мурлыкают?"
response, retrieved_docs = rag_pipeline(question, retriever, generator)

print(f"Вопрос: {question}")
print(f"Релевантные документы: {retrieved_docs}")
print(f"Ответ: {response}")

Загружено 8 фактов о кошках.


AttributeError: 'str' object has no attribute 'update'

In [29]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [34]:
# Установка необходимых библиотек
# pip install transformers torch datasets scikit-learn

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split

# 1. Загрузка датасета из файла cat-facts.txt
def load_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        facts = [line.strip() for line in f if line.strip()]
    return facts

dataset = load_dataset("кошки.txt")
print(f"Загружено {len(dataset)} фактов о кошках.")

# 2. Создание пар вопрос-ответ
def create_qa_pairs(facts):
    qa_pairs = []
    for fact in facts:
        # Простой способ создания вопросов: добавляем префикс к фактам
        question = f"Какие интересные факты о кошках вы знаете?"
        answer = fact
        qa_pairs.append({"question": question, "answer": answer})
    return qa_pairs

qa_pairs = create_qa_pairs(dataset)

# Разделение данных на обучающую и тестовую выборки
train_data, test_data = train_test_split(qa_pairs, test_size=0.1, random_state=42)

# 3. Подготовка данных для модели T5
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_data(data):
    inputs = [f"question: {item['question']} context: {item['answer']}" for item in data]
    targets = [item["answer"] for item in data]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
    labels = tokenizer(targets, max_length=150, truncation=True, padding="max_length", return_tensors="pt")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = Dataset.from_dict(preprocess_data(train_data))
test_dataset = Dataset.from_dict(preprocess_data(test_data))

# 4. Настройка модели T5
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# 5. Настройка параметров обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=40,
    weight_decay=0.01,
    save_steps=10_000,
    save_total_limit=2,
)

# 6. Обучение модели
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

# 7. Сохранение модели
model.save_pretrained("./finetuned_t5")
tokenizer.save_pretrained("./finetuned_t5")

# 8. Тестирование обученной модели
def generate_response(question, context, model, tokenizer):
    input_text = f"question: {question} context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=4, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Пример использования обученной модели
question = "сколько может спать кошка?"
context = "Кошки могут спать до 16 часов в день."
response = generate_response(question, context, model, tokenizer)
print(f"Вопрос: {question}")
print(f"Контекст: {context}")
print(f"Ответ: {response}")

Загружено 8 фактов о кошках.


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,11.233547
2,No log,9.658791
3,No log,8.258592
4,No log,6.959317
5,No log,5.720098
6,No log,4.591546
7,No log,3.564067
8,No log,2.626657
9,No log,1.823582
10,No log,1.184372


KeyboardInterrupt: 

In [37]:
print(qa_pairs)

[{'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'Сколько кошки мурлыкают?'}, {'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'кошки мурлыкают 8 часов'}, {'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'сколько глаз у кошки?'}, {'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'у кошек 2 глаза'}, {'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'сколько хвостов у кошки?'}, {'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'у кошки 1 хвост'}, {'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'чего много у кошек?'}, {'question': 'Какие интересные факты о кошках вы знаете?', 'answer': 'у кошек много добра'}]


In [35]:
model.save_pretrained("./finetuned_t5")
tokenizer.save_pretrained("./finetuned_t5")

('./finetuned_t5/tokenizer_config.json',
 './finetuned_t5/special_tokens_map.json',
 './finetuned_t5/spiece.model',
 './finetuned_t5/added_tokens.json')

In [46]:
question = "Какие интересные факты о кошках вы знаете?"
context = "у кошек много добра"
response = generate_response(question, context, model, tokenizer)
print(f"Вопрос: {question}")
print(f"Контекст: {context}")
print(f"Ответ: {response}")

Вопрос: Какие интересные факты о кошках вы знаете?
Контекст: у кошек много добра
Ответ: 
